# Criando um novo espaço vetorial baseado no dataset

## Importando os módulos necessários

In [ ]:
import pandas as pd
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow as tf

## Carregando modelo já criado em "model-making-script"

In [ ]:
searchModel = keras.models.load_model(
    '../model-making-scripts/searchModel.h5', custom_objects={'KerasLayer': hub.KerasLayer})

## Carregando o dataset e corrigindo entradas.

In [ ]:
df = pd.read_csv("./data/transcripts.csv")

del df['Unnamed: 0']

df['regulation'][0]

## Importando os dados para vetorização e criação do espaço vetorial

+ Serão importados na forma de Dataframe, usando Pandas
+ Após importação, é feita a concatenação das variáveis "num", "regulation" e "paragraph", pois juntas formarão o texto usado para produzir a representação vetorial do artigo.
+ Os campos são então armazenados na forma de listas, uma para cada coluna do Dataframe

In [ ]:

# Verifica se o campo 'paragraph' é nulo ou vazio
if df['paragraph'].isna().any() or df['paragraph'].str.strip().empty:
    df['Concatenation'] = df['num'].astype(str) + ' ' + df['regulation'].astype(str)
else:
    df['Concatenation'] = df['num'].astype(str) + ' ' + df['regulation'].astype(str) + ' ' + df['paragraph'].astype(str)

# Restante do código
list = df.values.tolist()
num = []
regulation = []
paragraph = []
sentences = []
for i in range(len(list)):
    regulation.append(list[i][0])
    num.append(list[i][1])
    paragraph.append(list[i][2])
    sentences.append(list[i][4])

df.head()


## Criando vetores

+ Para criar-se os vetores, o modelo é utilizado para codificar todos os itens da lista sentence
+ Esta lista, criada anteriormente, possui a concatenação do numero, regulamentação e paragrafo de cada artigo, na mesma ordem em que aparecem no dataset.

In [ ]:
vectors = searchModel.predict(sentences)

## Criando espaço vetorial
+ Os vetores criados anteriormente devem ser persistidos em JSON
+ O JSON possui diversos campos, cada um contendo um dado relevante acerca de um artigo.
+ Para cada vetor armazenado no campo "data", existe um metadado salvo em cada um dos outros campos no mesmo índice
+ Verificando-se um dado índice em todos os campos, é possível recuperar todas as informações de um dado.

In [ ]:
import json

lists = vectors.tolist()
data = {"data": lists,
        "num": num,
        "regulation": regulation,
        "paragraph": paragraph,
       }
json_str = json.dumps(data)

f = open('./data/data.json', 'w')
f.write(json_str)
f.close()